<img src="../images/logo.jpg" style="width:85px;height:85px;float:left" /><h1 style="position:relative;float:left;display:inline">Serving GANs in Production with Google Cloud Functions</h1>

<a href='https://colab.research.google.com/github/zurutech/gans-from-theory-to-production/blob/master/4.%20Production/4.1%20Serving%20Models%20using%20TF%202.0%20and%20Cloud%20Functions.ipynb'>
    <img align="left" src='https://cdn-images-1.medium.com/max/800/1*ZpNn76K98snC9vDiIJ6Ldw.jpeg'></img></a>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Serving-GANs-in-Production-with-Google-Cloud-Functions" data-toc-modified-id="Serving-GANs-in-Production-with-Google-Cloud-Functions-4">Serving GANs in Production with Google Cloud Functions</a></span><ul class="toc-item"><li><span><a href="#AI-Platform-vs-Cloud-Functions" data-toc-modified-id="AI-Platform-vs-Cloud-Functions-4.1">AI-Platform vs Cloud Functions</a></span><ul class="toc-item"><li><span><a href="#AI-Platform" data-toc-modified-id="AI-Platform-4.1.1">AI-Platform</a></span><ul class="toc-item"><li><span><a href="#Supported-Tensorflow-Versions" data-toc-modified-id="Supported-Tensorflow-Versions-4.1.1.1">Supported Tensorflow Versions</a></span></li></ul></li><li><span><a href="#Cloud-Functions" data-toc-modified-id="Cloud-Functions-4.1.2">Cloud Functions</a></span></li><li><span><a href="#Additional-Notes" data-toc-modified-id="Additional-Notes-4.1.3">Additional Notes</a></span></li></ul></li><li><span><a href="#Deploy-Models-Using-Google-Cloud-Functions" data-toc-modified-id="Deploy-Models-Using-Google-Cloud-Functions-4.2">Deploy Models Using Google Cloud Functions</a></span><ul class="toc-item"><li><span><a href="#Cold-vs-Warm-invocation" data-toc-modified-id="Cold-vs-Warm-invocation-4.2.1">Cold vs Warm invocation</a></span></li></ul></li><li><span><a href="#Writing-our-Cloud-function" data-toc-modified-id="Writing-our-Cloud-function-4.3">Writing our Cloud function</a></span><ul class="toc-item"><li><span><a href="#Requirements" data-toc-modified-id="Requirements-4.3.1">Requirements</a></span></li><li><span><a href="#main.py" data-toc-modified-id="main.py-4.3.2">main.py</a></span><ul class="toc-item"><li><span><a href="#Imports-and-global-variables" data-toc-modified-id="Imports-and-global-variables-4.3.2.1">Imports and global variables</a></span></li><li><span><a href="#Preliminaries---Model-Format" data-toc-modified-id="Preliminaries---Model-Format-4.3.2.2">Preliminaries - Model Format</a></span></li><li><span><a href="#Model-definition" data-toc-modified-id="Model-definition-4.3.2.3">Model definition</a></span></li><li><span><a href="#Download-weights" data-toc-modified-id="Download-weights-4.3.2.4">Download weights</a></span></li><li><span><a href="#Handler---The-Cloud-Function-Entry-Point" data-toc-modified-id="Handler---The-Cloud-Function-Entry-Point-4.3.2.5">Handler - The Cloud Function Entry Point</a></span></li></ul></li></ul></li><li><span><a href="#Deploying-our-Cloud-function" data-toc-modified-id="Deploying-our-Cloud-function-4.4">Deploying our Cloud function</a></span><ul class="toc-item"><li><span><a href="#Save-model-weights-on-Google-Cloud-Storage" data-toc-modified-id="Save-model-weights-on-Google-Cloud-Storage-4.4.1">Save model weights on Google Cloud Storage</a></span></li><li><span><a href="#Deploy-Google-Cloud-Function-code" data-toc-modified-id="Deploy-Google-Cloud-Function-code-4.4.2">Deploy Google Cloud Function code</a></span><ul class="toc-item"><li><span><a href="#Command-Line-Deployment" data-toc-modified-id="Command-Line-Deployment-4.4.2.1">Command Line Deployment</a></span></li><li><span><a href="#Dashboard-Deployment" data-toc-modified-id="Dashboard-Deployment-4.4.2.2">Dashboard Deployment</a></span></li><li><span><a href="#Testing" data-toc-modified-id="Testing-4.4.2.3">Testing</a></span></li><li><span><a href="#Logging" data-toc-modified-id="Logging-4.4.2.4">Logging</a></span></li></ul></li></ul></li><li><span><a href="#Demo" data-toc-modified-id="Demo-4.5">Demo</a></span></li></ul></li></ul></div>

![Google Cloud Logo](images/google-cloud.png)


![Google CloudML Engine Logo](https://storage.googleapis.com/gweb-cloudblog-publish/images/CloudFunctions_TF.max-2200x2200.png)

**Disclaimer**: highly inspired from this google [blog post](https://cloud.google.com/blog/products/ai-machine-learning/how-to-serve-deep-learning-models-using-tensorflow-2-0-with-cloud-functions).

**Tensorflow 2.0** offers a number of enhancements. However, the new framework brings some challenges, like how to **deploy** TF 2.0 models. [Google Cloud Functions](https://cloud.google.com/functions/) offer a convenient way of running inference within Google Cloud infratructure allowing you to run the most recent version of this framework.

## AI-Platform vs Cloud Functions

AI-Platform and Google Cloud Functions are the two best ways for deploying models.

### AI-Platform

[AI-Platform](https://cloud.google.com/ml-engine/) (previously ML-Engine):

> Google Cloud Machine Learning (ML) Engine is a managed service that enables developers and data scientists to build and bring superior machine learning models to production. Cloud ML Engine offers __training__ and __prediction__ services, which can be used together or individually.

**Features:**
- __Codeless__ inference
- __Scalable__ infrastructure
- __Managed__ Service
- Separate storage of the model (and __versioning__)

#### Supported Tensorflow Versions

Taken from: https://cloud.google.com/ml-engine/docs/runtime-version-list (August 2019)
![Versions](images/tf-versions-ml-engine.png)

__As we can see the last TF version supported is 1.14 so there is no way by now to deploy our TF 2.0 model using AI- Platform__.

### Cloud Functions

[Google Cloud Functions](https://cloud.google.com/functions/):

> Google Cloud Functions is a lightweight compute solution for developers to create single-purpose, __stand-alone functions__ that respond to __Cloud events__ without the need to manage a server or runtime environment.

**Serverless approach:**

> Cloud Functions removes the work of managing servers, configuring software, updating frameworks, and patching operating systems. The software and infrastructure are __fully managed__ by Google so that you __just add code__. Furthermore, provisioning of resources happens automatically in response to events. This means that a function can __scale__ from a few invocations a day to many millions of invocations __without any work__ from you.

**Features:**

- Simple code for implementing inference that at the same time allows you to implement __custom logic__.
- Great __scalability__
- Pay for runs (you don't pay for idle servers)
- __Custom versions__ of different frameworks (Tf 2.0 or PyTorch)

![Differences](images/differences-cloud-ai.png)

### Additional Notes
- Google provides some __free invocations__ of GCF per month, so this kind of setup might be perfect for a toy project
- The model, as we will see later needs to be downloaded when the cloud function is in **cold start** phase, this may take a while for big models
- You can use Cloud Functions in combination with AI Platform Predictions (you can learn more in this [post](https://cloud.google.com/blog/products/ai-machine-learning/empower-your-ai-platform-trained-serverless-endpoints-with-machine-learning-on-google-cloud-functions))
- Google Cloud Functions have __limited storage__ (see [here](https://cloud.google.com/functions/quotas)), so in our cases we need to upload our model to Google Cloud Storage and then download it during the first invocation of the function

## Deploy Models Using Google Cloud Functions

Our architecture and approach is very simple.
We will __train__ the model **locally** and then upload it to **Google Cloud**. Cloud Functions will be invoked through an API request and will **download** model, **generate** an image and **answer** with the generated image.

![Architecture](https://storage.googleapis.com/gweb-cloudblog-publish/images/cloud_function_Architecture_Overview.max-1000x1000.png)

### Cold vs Warm invocation

**Cold invocation:** The very first cold start happens when the __first request__ comes in after deployment. After that request is processed, the instance stays alive to be reused for subsequent requests. There is no predefined threshold for instance recycling. Remember: in the first invocation we also need to __download__ our model.

**Warm invocation:** After the first invocation every other invocation is a **warm** one. The model is already downloaded and we will only perform inference.

## Writing our Cloud function

### Requirements

One of the main __upsides__ of Cloud Functions is that you don't have to manually generate the package. You can just use a __requirements.txt__ file and list all used libraries there.

Let's see our __requirements.in__ file:

```
numpy
tensorflow==2.0.0rc0
Pillow
google-cloud-storage

# Dev
Flask
black
pylint
flake8
flake8-bugbear
isort
mypy
pytest
pytest-cov
pydocstyle
opencv-python
```

To obtain the correct __requirements.txt__ file we use the `pip-compile` tool, from `pip-tools`. We just need to run the command:

```
pip-compile requirements.in
```

and we obtain our __requirements.txt__ file.



### main.py

#### Imports and global variables
Remember to have the model as a **global variable** in your python code so it will be **cached** and **reused** in **warm** invocations of Cloud Functions.

In [ ]:
"""
Serving a TF 2.0 Model via Cloud Function.

More info: http://bit.ly/310wRZl
"""
import base64  # needed for image encoding
import os
import tempfile
from io import BytesIO
from typing import TYPE_CHECKING, List

import numpy as np
from PIL import Image
import tensorflow as tf
from google.cloud import storage

from flask import Request

# Download model configuration
DOWNLOAD_CONFIG = {
    "bucket_name": "euroscipy-2019-workshop",  # name of the bucket
    "model_id": "dcgan-weights",  # name of the model
    "destination_folder": tempfile.gettempdir(),  # tmp directory
}

# Header definitions needed for the response (Needed for CORS)
# CORS: https://developer.mozilla.org/en-US/docs/Web/HTTP/CORS)
headers = {
    "Access-Control-Allow-Headers": "Content-Type",
    "Access-Control-Allow-Origin": "*",
}

# Model variables

# Dimension of the latent space of the model
LATENT_DIMENSION = 100

# We keep model as global variable so we don't have to reload it in case of warm invocations
MODEL: tf.keras.Model = None

#### Preliminaries - Model Format

The first thing to do is to __define__ and __download__ our model from Google Cloud Storage.

**Weight format, not SavedModel**

It would be great to load our model as [SavedModel](https://www.tensorflow.org/beta/tutorials/keras/save_and_restore_models) inside our Cloud Function.
However the SavedModel format is __not__ compatible with the Cloud Functions storage system.

__SavedModel__ format:

```
- model
|
| ----- saved_model.pb
| ----- variables (directory)
```

However inside our Cloud Function we can __only__ write inside the `tmp` directory and we cannot create any directory inside `tmp`, so we **cannot** download a SavedModel.

We need to download only the model's __weights__ after having defined our model inside the function.

We can download the model weights since the __Weight format__ is:

```
- model
|
| ----- weights.data-**-of**
| ----- weights.index
```

Here we do **not** have nested folders so we can simply download our weights inside the `tmp` directory.

__Remember__: to export weights from a keras model `G` you just need to run:

```python
G.save_weights('generator')
```

#### Model definition

Since we need to download weights we need to define our model inside the Cloud Function:

In [ ]:
# IMPORTANT: same definition as the previous notebook
def get_model():
    G = tf.keras.Sequential(
        [
            tf.keras.layers.Dense(
                1024 * 4 * 4, use_bias=False, input_shape=(LATENT_DIMENSION,)
            ),
            tf.keras.layers.Reshape((4, 4, 1024)),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(),
            tf.keras.layers.Conv2DTranspose(
                256, (5, 5), strides=(2, 2), padding="same", use_bias=False
            ),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(),
            tf.keras.layers.Conv2DTranspose(
                128, (5, 5), strides=(2, 2), padding="same", use_bias=False
            ),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(),
            tf.keras.layers.Conv2DTranspose(
                64, (5, 5), strides=(2, 2), padding="same", use_bias=False
            ),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(),
            tf.keras.layers.Conv2DTranspose(
                3, (5, 5), strides=(2, 2), padding="same", use_bias=False
            ),
            tf.keras.layers.Activation(tf.math.tanh),
        ]
    )
    return G

#### Download weights

Weights can be downloaded using this utility function:

In [ ]:
def download_blobs(model_id: str, destination_folder: str, bucket_name: str) -> str:
    """
    Download all the models related blobs from the bucket.

    Args:
        model_id (str): ID of the model we want to download AKA the name of the folder on
            GCS. This is used as a prefix filtering out all unneeded blobs.
        destination_folder (str): Path of the destination folder where blobs will be downloaded.
        bucket_name (str): Name of the bucket to use as target storage.

    Returns:
        :py:obj:`list` of [:py:obj:`str`]: Returns a list containing the path of the stored SavedModel(s).
    """
    weights_uri: str = ""  # uri of the index file for weights

    prefix = model_id

    # Instantiate a google storage client
    storage_client = storage.Client()

    # get the bucket we are interested in
    bucket = storage_client.get_bucket(bucket_name)

    # List blobs iterate in folder
    blobs = bucket.list_blobs(prefix=prefix)  # Excluding folder inside bucket
    for blob in blobs:
        print("Downloading ", blob.name)
        blob_name = blob.name.split(os.path.sep)[-1]
        if blob_name != "":
            destination_uri = os.path.join(destination_folder, blob_name)
            # We are downloading the blob of a file
            try:
                blob.download_to_filename(destination_uri)
                if blob.name.endswith(".index"):
                    weights_uri = destination_uri
            except IsADirectoryError:
                # We cannot download the blob of a folder
                continue

    if weights_uri == "":
        raise ValueError(f"No index file found in {bucket_name}/{model_id}")

    # return the uri of the weights, needed for loading weights
    return weights_uri

#### Handler - The Cloud Function Entry Point

The __handler__ is the function triggered when a request arrives. This is the __entry point__ of our Google Cloud Function.  
Here we can implement our custom logic, adding pre or post-processing.

We need to perform the following steps in our handler:

- __Download model__ (if needed)
- __Load weights__ (if needed)
- __Inference__ (`model.call()`)
- __Post-Processing__
- __Compose Response__ for the client
    
In our example we use the generator defined before to generate an image.  
We support the generation from a latent vector (we will see later why) and a generation using a new latent vector.
So, in our function we need to parse the request, searching for a latent vector, if the latent vector is not present we generate a new latent vector (`tf.random.normal((1, LATENT_DIMENSION))`).  
We call the model using the latent vector as input. Then we scale correctly the output of the model and return the encoded representation of the image.

We need to define our post-processing function that encodes the image:

In [ ]:
def postprocess(output: np.ndarray) -> str:
    """
    Post process the model's output
    Args:
        output: output image to process

    Returns:
        The image encoded as str (base64)
    """
    pil_img = Image.fromarray(output)
    buff = BytesIO()
    image_format = "PNG"
    pil_img.save(buff, format=image_format)
    encoded_img = base64.b64encode(buff.getvalue()).decode("utf-8")
    return encoded_img

And finally our entry point:

In [ ]:
def handler(request: Request = None):
    """
    Entry point of the Serveless call.

    Args:
        request (:py:class:`flask.Request`): Flask Request holding our payload.

    """
    global MODEL
    # Model load which only happens during cold starts
    if not MODEL:
        # The model is not defined, we need to instantiate it and download its weights
        print("Cold start: Loading model")
        print("Downloading weights")
        weights_path = download_blobs(**DOWNLOAD_CONFIG)

        # instantiate the model
        MODEL = get_model()

        # load weights
        MODEL.load_weights(weights_path.replace(".index", ""))

        print("weights loaded")
    else:
        # the model is already defined, we are in warm start phase
        print("Warm start: Using cached model")

    # log the request
    print("Received", request)

    # get the request payload
    request = request.get_json(silent=True)

    # get the noise vector if present
    try:

        # use the fed noise
        noise = tf.constant(np.array(request["noise_vector"]))

        # check the correct dimensions
        if noise.shape != (1, LATENT_DIMENSION):
            return ({}, 422, headers)

        print("Using fed noise")
    except Exception:
        # the noise vector is not present, we need to generate a new vector
        print("No noise vector provided, generating noise")
        noise = tf.random.normal((1, LATENT_DIMENSION))

    # call the model
    output = MODEL.call(noise).numpy().squeeze()

    # back in the rage [0, 255]
    output = ((output + 1) * 127.5).astype(np.uint8)

    # postprocess stage (encode)
    encoded_image = postprocess(output)

    # compose the output response
    return (
        {
            "base64_image": encoded_image,
            "format": "png",
            "noise_vector": noise.numpy().tolist(),
        },
        200,
        headers,
    )

## Deploying our Cloud function

### Save model weights on Google Cloud Storage

After the `G.save_weights('generator')` command of the previous notebook you should have the weights of the generator network inside the directory `3. AshPy`. We need to copy the network weights to Google Cloud Storage to have them available from the Cloud Function.

Upload the weights using the `gsutil` command line tool:

```bash
cd 3.\ AshPy
mkdir dcgan
cp generator* dcgan
gsutil cp -r dcgan gs://euroscipy-2019-workshop/dcgan-weights
```

After these commands we will find the model weights inside the bucket `euroscipy-2019-workshop/dcgan-weights`.


### Deploy Google Cloud Function code

#### Command Line Deployment

Reference: [Gcloud Function Deploy](https://cloud.google.com/sdk/gcloud/reference/functions/deploy).

Now we need to deploy our Cloud Function.
We can easily deploy and run Cloud Functions using `gcloud`:

```bash
cd 4.\ Production/deploy
gcloud functions deploy euroscipy-2019-tutorial --entry-point handler --stage-bucket zuru-ml-cloud-functions --runtime python37 --trigger-http --memory 2048
```

Parameters:
- `euroscipy-2019-tutorial`: ID of the function or fully qualified identifier for the function.
- `--entry-point handler`: Name of a Google Cloud Function (as defined in source code) that will be executed. 
- `--stage-bucket zuru-ml-cloud-functions`: When deploying a function from a local directory, this flag's value is the name of the Google Cloud Storage bucket in which source code will be stored.
- `--runtime python37`: Runtime in which to run the function.
- `--trigger-http`: Function will be assigned an endpoint, which you can view by using the describe command.
- `--memory 2048`: Limit on the amount of memory the function can use.

#### Dashboard Deployment

You can also upload the function from the [dashboard](https://console.cloud.google.com/functions/list?_ga=2.247640449.-709977754.1558685959)

![Dashboard](https://storage.googleapis.com/gweb-cloudblog-publish/images/cloud_functions_create_function.max-1200x1200.png)

Next, set the main.py and requirements.txt files.

Finally, push the “Create” button and initialize the creation of the function.


#### Testing

Once the function is deployed, you can test it in the “Testing” section of the Cloud Functions dashboard.
You can also customize incoming events and see output as well as logs.

![Testing](images/testing.png)


#### Logging

Read logs from the command line:

```bash
gcloud functions logs read euroscipy-2019-tutorial

```

## Demo

Try our demo at: http://zurutech.github.io/gans-from-theory-to-production.  
Interpolation demo: http://zurutech.github.io/gans-from-theory-to-production/interpolation.html